In [28]:
import gurobipy as grb

In [29]:
import random

In [30]:
# declare n , m here

# Number of students
n = 20

# Number of projects
m = 10

In [31]:
# Weights in the objective function

alpha = 0.5
beta = 0.5

In [32]:
def create_friend_groups(n,m):
    G = []
    for a1 in range(n):
        temp = [0]*n
        for a2 in range(n):
            if(a1//5 == a2//5):
                temp[a2] = 1
        G.append(temp)
    return G

def friend_groups_have_same_project(n,m):
  H = []
  for a in range(n):
    temp = [0]*m
    temp[a//5] = 1
    H.append(temp)
  return H

In [33]:
# Generate the social network matrix

G = []
for i in range(n):
  randomlist = []
  for j in range(n):
    if i==j:
      randomlist.append(0)
    else:
      randomlist.append(random.choice([-1,0,1]))
        # randomlist.append(1)
  G.append(randomlist)
# G = create_friend_groups(n,m)

In [34]:
# Generate the project preference matrix

H = []
for i in range(n):
  randomlist = []
  for j in range(m):
    rand_int = random.randint(0,1)
    randomlist.append(rand_int)
  H.append(randomlist)
# H = friend_groups_have_same_project(n, m)

In [35]:
# Generate the project list input

# P = []

# while(1):
#   P = []
#   temp2 = 0
#   for i in range(m):
#     # t1 is the upper bound for the number of teams that can work on that project
#     # t3 is the upper bound for the team size of any of those teams
#     (t1,t3) = (random.randint(1,3), random.randint(3,6))
#     temp2 += t1*t3
#     P.append((t1,t3))
#   if(temp2 >= n):
#     break

P = [(1,5)]*m

# P = [(1,2), (1,3), (1,3)]

print(P)

[(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]


In [36]:
GRB = grb.GRB


In [37]:

model =grb.Model("Cluster")

In [38]:
y_indices = []
for i in range(m):
    for t in range(P[i][0]):
        for a in range(n):
            y_indices.append((i,t,a))

# print(y_indices)
print(len(y_indices))

200


In [39]:
y = model.addVars(y_indices,vtype=GRB.BINARY,name="y")
print(len(y))

200


In [40]:
l_indices = []
for i in range(m):
    for t in range(P[i][0]):
        for a1 in range(n):
            for a2 in range(n):
                l_indices.append((i,t,a1,a2))

# print(l_indices)
print(len(l_indices))

4000


In [41]:
l = model.addVars(l_indices,vtype=GRB.BINARY,name="l")
print(len(l))

4000


In [42]:
# for a in range(n):
#     temp = 0
#     for i in range(m):
#         for t in range(P[i][0]):
#             temp += y[(i,t,a)]
for a in range(n):
    model.addConstr( 1 == sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )
    

In [43]:
for i in range(m):
    for t in range (P[i][0]):
        for a1 in range(n):
            for a2 in range(n):
                # model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a1)])
                # model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a2)])
                # model.addConstr(l[(i,t,a1,a2)]>= y[(i,t,a1)]+y[(i,t,a2)]-1)
                model.addConstr(l[(i,t,a1,a2)] == y[(i,t,a1)]*y[(i,t,a2)])

In [44]:
for i in range(m):
    for t in range (P[i][0]):
        model.addConstr( P[i][1] >= sum(y[(i,t,a)] for a in range(n)))

In [45]:
model.setObjective(sum((y[(i,t,a)]*H[a][i]) for i in range(m) for t in range (P[i][0]) for a in range (n)) + 
sum((l[(i,t,a1, a2)]*G[a1][a2]) for i in range(m) for t in range (P[i][0]) for a1 in range (n) for a2 in range(n) if (a1!=a2))
,GRB.MAXIMIZE)

In [46]:
# model.setParam("NodeMethod",2)
# model.setParam("Method",5)
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 4200 columns and 400 nonzeros
Model fingerprint: 0xc8c43d7e
Model has 4000 quadratic constraints
Variable types: 0 continuous, 4200 integer (4200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 0 rows and 2100 columns
Presolve time: 0.01s
Presolved: 7630 rows, 4000 columns, 17500 nonzeros
Variable types: 0 continuous, 4000 integer (4000 binary)
Found heuristic solution: objective 9.0000000
Found heuristic solution: objective 22.0000000

Root relaxation: objective -8.900000e+01, 1219 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

In [51]:
# for v in model.getVars():
#         print(v.varName , v.X)
#      if v.x!=0.0:
#             print(v.varName, v.x)
print('Obj:', model.objVal)

x_= model.getVarByName("y")




Obj: 52.0


TypeError: 'NoneType' object is not iterable